In [33]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2159,  0.0860,  0.1895,  0.2205, -0.0622,  0.0052,  0.1583,  0.0288,
         -0.1752, -0.0731],
        [ 0.2068,  0.1539,  0.0654,  0.1765, -0.1418, -0.0421,  0.1995,  0.0191,
         -0.1000, -0.2010]], grad_fn=<AddmmBackward0>)

In [34]:
# 自定义块
class MLP(nn.Module):
    # 用模型参数声明层
    def __init__(self):
        # 执行必要的初始化
        super().__init__() # 调用父类初始化（__init__）函数
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播：如何根据输入X返回所需的模型输出 
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[-0.0991,  0.1330,  0.0617,  0.0336,  0.1230, -0.1006, -0.0264, -0.1258,
         -0.0209, -0.0821],
        [-0.1998,  0.2116,  0.0311,  0.0243,  0.1868, -0.1500, -0.0100, -0.0787,
         -0.0093, -0.2808]], grad_fn=<AddmmBackward0>)

In [35]:
# 自定义顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # module为Module子类的一个实例，
            # 将它保存在‘Module’类的成员变量_modules中，_modules类型为OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0779,  0.3066, -0.0933,  0.1324, -0.0879, -0.1174, -0.0982, -0.1194,
          0.0004, -0.0504],
        [ 0.2389,  0.2825, -0.1162,  0.1765, -0.0617, -0.1880, -0.1164, -0.1670,
          0.0308, -0.0489]], grad_fn=<AddmmBackward0>)

In [36]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，相当于两个全连接层共享参数
        X = self.linear(X)
        
        # 插入python控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0621, grad_fn=<SumBackward0>)

In [37]:
# 嵌套块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(
    NestMLP(), 
    nn.Linear(16, 20),
    FixedHiddenMLP(),
)

chimera(X)

tensor(0.3098, grad_fn=<SumBackward0>)